###### Name: Mysara Qistina binti Mahadzir
###### Student ID: SW01083524

##### The chosen e-commerce platform is Shopee

In [1]:
import undetected_chromedriver as uc
import pickle
import time
import bs4
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Start undetected Chrome with custom options
options = uc.ChromeOptions()
options.add_argument(r"C:\Users\user\AppData\Local\Google\Chrome\User Data")  # Path to Chrome User Data
options.add_argument("--profile-directory=Default")  # Use the default profile
options.add_argument("--disable-blink-features=AutomationControlled")  # Disable automation detection
options.add_argument("--start-maximized")  # Start with a maximized window

# Initialize the Chrome driver with options
driver = uc.Chrome(options=options)

# Open Shopee product page and wait for login
url = "https://shopee.com.my/Apple-MacBook-Air-(13-inch-M1)-i.1278832578.28854289815"
driver.get(url)
time.sleep(60)  # Wait for user to login manually

# Scroll to load reviews on the product page
driver.execute_script("window.scrollBy(0, 700);")  # Scroll the page down
time.sleep(5)  # Wait for page to load more reviews

# Initialize list to store reviews data
reviews = []

# Loop through multiple pages of reviews
for k in range(5):  # Scrape reviews from 5 pages
    time.sleep(3)  # Wait for content to load
    soup = bs4.BeautifulSoup(driver.page_source, "html.parser")  # Parse page source using BeautifulSoup

    # Extract review details (name, date, and content) for each review
    for r in soup.find_all("div", class_="shopee-product-rating"):
        name = r.find("a", class_="shopee-product-rating__author-name")  # Find reviewer name
        date = r.find("div", class_="shopee-product-rating__time")  # Find review date
        content = r.find("div", style="margin-top: 0.75rem;")  # Find review content

        # Clean the review date
        if date:
            full_text = date.text.strip()
            clean_date = full_text.split(" | ")[0]  # Extract date before separator
        else:
            clean_date = "No Date"  # If no date available, set as 'No Date'
        
        # Append review data (name, date, content) to the reviews list
        reviews.append([
            name.text.strip() if name else "No Name",  # If name exists, strip whitespace
            clean_date,  # Cleaned review date
            content.text.strip() if content else "No Content"  # If content exists, strip whitespace
        ])

    # Try to click the next page button to load more reviews
    try:
        next_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.shopee-icon-button--right"))
        )
        next_btn.click()  # Click the next page button
    except:
        print("No more pages.")  # If no next page is available, exit the loop
        break

# Close the browser after scraping
driver.quit()

# Save scraped reviews to a CSV file
df = pd.DataFrame(reviews, columns=["Reviewer Name", "Review Date", "Review Content"])
df.to_csv("extracted_review.csv", index=False)

# Print success message
print("Data saved as 'extracted_review.csv'.")

Data saved as 'extracted_review.csv'.
